<a href="https://colab.research.google.com/github/TaraRasti/Deep_Learning_F20_Assignments/blob/master/Assignment%208/dropout/FashionMNIST_Adam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable
from sklearn.model_selection import train_test_split

In [3]:
train = pd.read_csv('/content/drive/MyDrive/fashion-mnist_train.csv')
print(train.shape)

(60000, 785)


In [4]:
test = pd.read_csv('/content/drive/MyDrive/fashion-mnist_test.csv')
print(test.shape)

(10000, 785)


In [5]:
y_train = train['label'].values
X_train = train.drop(['label'],1).values
y_test = test['label'].values
X_test = test.drop(['label'],1).values
print('y test shape : ',y_test.shape)
print('X test shape : ',X_test.shape)
print('y train shape : ',y_train.shape)
print('X train shape : ',X_train.shape)

y test shape :  (10000,)
X test shape :  (10000, 784)
y train shape :  (60000,)
X train shape :  (60000, 784)


In [9]:
BATCH_SIZE = 32

torch_X_train = torch_X_train.view(-1, 1,28,28).float()
torch_X_test = torch_X_test.view(-1,1,28,28).float()
print(torch_X_train.shape)
print(torch_X_test.shape)

train = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)
test = torch.utils.data.TensorDataset(torch_X_test,torch_y_test)

train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = BATCH_SIZE, shuffle = False)

torch.Size([60000, 1, 28, 28])
torch.Size([10000, 1, 28, 28])


In [12]:
def fit(model, train_loader):
    optimizer = torch.optim.Adam(model.parameters())
    error = nn.CrossEntropyLoss()
    EPOCHS = 5
    model.train()
    for epoch in range(EPOCHS):
        correct = 0
        for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
            var_X_batch = Variable(X_batch).float()
            var_y_batch = Variable(y_batch)
            optimizer.zero_grad()
            output = model(var_X_batch)
            loss = error(output, var_y_batch)
            loss.backward()
            optimizer.step()

         
            predicted = torch.max(output.data, 1)[1] 
            correct += (predicted == var_y_batch).sum()
      
            if batch_idx % 50 == 0:
                print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                    epoch, batch_idx*len(X_batch), len(train_loader.dataset), 100.*batch_idx / len(train_loader), loss.item(), float(correct*100) / float(BATCH_SIZE*(batch_idx+1))))

In [13]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5)
        self.conv3 = nn.Conv2d(32,64, kernel_size=5)
        self.fc1 = nn.Linear(3*3*64, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.dropout(x, p=0.2, training=self.training)
        x = F.relu(F.max_pool2d(self.conv3(x),2))
        x = F.dropout(x, p=0.3, training=self.training)
        x = x.view(-1,3*3*64 )
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
 
cnn = CNN()
print(cnn)

it = iter(train_loader)
X_batch, y_batch = next(it)
print(cnn.forward(X_batch).shape)

CNN(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)
torch.Size([32, 10])


In [14]:
fit(cnn,train_loader)

Epoch : 0 [0/60000 (0%)]	Loss: 13.644171	 Accuracy:3.125%
Epoch : 0 [1600/60000 (3%)]	Loss: 1.173018	 Accuracy:35.968%
Epoch : 0 [3200/60000 (5%)]	Loss: 1.242868	 Accuracy:50.495%
Epoch : 0 [4800/60000 (8%)]	Loss: 0.712533	 Accuracy:56.623%
Epoch : 0 [6400/60000 (11%)]	Loss: 0.926103	 Accuracy:60.370%
Epoch : 0 [8000/60000 (13%)]	Loss: 1.467777	 Accuracy:62.674%
Epoch : 0 [9600/60000 (16%)]	Loss: 0.562799	 Accuracy:64.545%
Epoch : 0 [11200/60000 (19%)]	Loss: 0.668852	 Accuracy:66.026%
Epoch : 0 [12800/60000 (21%)]	Loss: 0.949386	 Accuracy:67.051%
Epoch : 0 [14400/60000 (24%)]	Loss: 0.967023	 Accuracy:68.057%
Epoch : 0 [16000/60000 (27%)]	Loss: 0.554034	 Accuracy:68.900%
Epoch : 0 [17600/60000 (29%)]	Loss: 0.518291	 Accuracy:69.657%
Epoch : 0 [19200/60000 (32%)]	Loss: 0.479473	 Accuracy:70.133%
Epoch : 0 [20800/60000 (35%)]	Loss: 0.435681	 Accuracy:70.709%
Epoch : 0 [22400/60000 (37%)]	Loss: 0.603964	 Accuracy:71.238%
Epoch : 0 [24000/60000 (40%)]	Loss: 0.511564	 Accuracy:71.671%
Epoch 

In [16]:
def evaluate(model):
    correct = 0 
    for test_imgs, test_labels in test_loader:
        test_imgs = Variable(test_imgs).float()
        output = model(test_imgs)
        predicted = torch.max(output,1)[1]
        correct += (predicted == test_labels).sum()
    print("Test accuracy:{:.3f}% ".format( float(correct) / (len(test_loader)*BATCH_SIZE)))

In [17]:
evaluate(cnn)

Test accuracy:0.849% 
